<a href="https://colab.research.google.com/github/kferrel-shp/Project_Codes/blob/main/Forcing_data_analysis_conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ***The purpose of this code is to convert the PRISM precip data processing python code into a .ipynb format that can be processed using google colab.***

# I'm doing this because in order for me to get the precipitation data that I need, I have to run this code block BUT my python IDE is not allowing me to download some of the necessary imports, so I have to improvise.

# **Shout out to [Cassie Zhang](https://snre.arizona.edu/cassie-zhang) with the University of Arizona for originally writing this script. I rewrote it in a way that it can be in a .ipynmb format and add additional comments when needed.**

# **Imports**

In [ ]:
# !pip install rasterstats

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 86.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 10.8 MB/s eta 0:00:00


In [54]:
from google.colab import drive
import os
import glob
import geopandas as gpd
import csv
import pandas as pd
from rasterstats import zonal_stats
from concurrent.futures import ProcessPoolExecutor

# **Setting up drive directory**

In [ ]:
#drive.mount('/content/drive') # allows colab to access my google drive. Only need to run it once

Mounted at /content/drive


In [55]:
base_dir = '/content/drive/My Drive/prism_precip' # base path where everything is located
tif_dir = os.path.join(base_dir, 'precip/ppt') # directory where tif files are located
Tucson_cruz = os.path.join(base_dir, 'watersheds/Tucson_cruz/Tuc_Cruz_GCS.shp') # directory where tucson_cruz.shp is located

In [56]:
assert os.path.exists(tif_dir) # check if tif_dir exist. If not, error will occur
assert os.path.exists(Tucson_cruz) # check if shapefile_dir exist. If not, error will occur

# **Reading in shapefile and checking how many features are in the shapefile**

In [57]:
tuc_cruz_map = gpd.read_file(Tucson_cruz) # read in shapefile
tuc_cruz_features = tuc_cruz_map['Subbasin'] # access the attribute table. Specifically the column called 'Subbasin'
print('There are in total ' + str(len(tuc_cruz_features)) + ' subbasins in the shapefile') # there should be 25 total features

There are in total 25 subbasins in the shapefile


In [67]:
                                                                      # output locations

base_output = '/content/drive/My Drive/prism_precip/Outputs' # output directory
tuc_cruz_output_file = os.path.join(base_output, 'Tuc_Cruz/tucson_cruz_output_2000_2010.csv') # output directory for the Tucson Cruz watershed

assert os.path.exists(base_output) # check if base_output exist. If not, error will occur
#os.makedirs(base_output, exist_ok=True) # create output directory if it doesn't exist

# **Processing Function**

In [68]:
def Process_tif_files(filename):
    cur_tsr = filename.split('/')[-1][-14:-4]  # Extract date from filename
    cur_tif = filename

    #means = zonal_stats(tuc_cruz_features, cur_tif, stats="mean")
    #cur_means = [x['mean'] for x in means]
    #cur_line = [cur_tsr] + cur_means

    #geojson_out=True ensures that zonal_stats returns a list of GeoJSON Feature objects with the calculated statistics.
    means = zonal_stats(tuc_cruz_map, cur_tif, stats="mean", geojson_out=True)

    cur_means = [x['properties']['mean'] for x in means] #extract the mean from the properties in GeoJSON
    cur_line = [cur_tsr] + cur_means
    print(f'Finished Processing {filename}')
    return cur_line

# **Main Execution Block**

In [69]:
if __name__ == '__main__':
    filenames = []
    st_year = 2000  # Start year
    end_year = 2010  # End year

    # ---- List all files within the folder ----
    for cur_year in range(st_year, end_year + 1):
        cur_folder = os.path.join(tif_dir, str(cur_year))
        if os.path.exists(cur_folder):  # Ensure the folder exists
            for file in glob.glob(os.path.join(cur_folder, "*.tif")):
                filenames.append(file)
        else:
            print(f"Warning: Folder {cur_folder} not found!")

    if len(filenames) == 0:
        raise ValueError("No TIF files found! Check file paths.")

    # ---- Parallel processing ----
    with ProcessPoolExecutor() as executor:
        results = executor.map(Process_tif_files, filenames)

    # ---- Save results in CSV ----
    with open(tuc_cruz_output_file, 'w', newline='') as csvfile:
        Qwriter = csv.writer(csvfile)
        Qwriter.writerow(['Time'] + tuc_cruz_features.values.tolist())

        for result in results:
            try:
                Qwriter.writerow(result)
            except Exception as e:
                print(f"Error writing row: {result}, Error: {e}")
                continue

    print(f"Processing complete! CSV saved at {tuc_cruz_output_file}")

Finished Processing /content/drive/My Drive/prism_precip/precip/ppt/2000/2000-01-07.tif
Finished Processing /content/drive/My Drive/prism_precip/precip/ppt/2000/2000-01-09.tif
Finished Processing /content/drive/My Drive/prism_precip/precip/ppt/2000/2000-01-10.tif
Finished Processing /content/drive/My Drive/prism_precip/precip/ppt/2000/2000-01-24.tif
Finished Processing /content/drive/My Drive/prism_precip/precip/ppt/2000/2000-01-19.tifFinished Processing /content/drive/My Drive/prism_precip/precip/ppt/2000/2000-01-28.tif

Finished Processing /content/drive/My Drive/prism_precip/precip/ppt/2000/2000-01-06.tifFinished Processing /content/drive/My Drive/prism_precip/precip/ppt/2000/2000-01-02.tif

Finished Processing /content/drive/My Drive/prism_precip/precip/ppt/2000/2000-01-27.tif
Finished Processing /content/drive/My Drive/prism_precip/precip/ppt/2000/2000-01-17.tif
Finished Processing /content/drive/My Drive/prism_precip/precip/ppt/2000/2000-01-01.tif
Finished Processing /content/dri